In [ ]:
import os
import gglasso
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt

from numpy.linalg import matrix_rank
from matplotlib.pyplot import figure
from scipy import stats
from scipy.linalg import eigh
from numpy import genfromtxt

from gglasso.solver.admm_solver import ADMM_MGL
from gglasso.problem import glasso_problem

from gglasso.helper.data_generation import generate_precision_matrix, group_power_network, sample_covariance_matrix
from gglasso.helper.basic_linalg import adjacency_matrix
from gglasso.helper.data_generation import time_varying_power_network, sample_covariance_matrix
from gglasso.helper.experiment_helper import lambda_grid, discovery_rate, error
from gglasso.helper.utils import get_K_identity
from gglasso.helper.experiment_helper import plot_evolution, plot_deviation, surface_plot, single_heatmap_animation
from gglasso.helper.model_selection import aic, ebic, K_single_grid

In [ ]:
def read_sub_corr(K=int, path=str):
    sub_corr = []

    for i in range(0, K):
        sub_corr.append(genfromtxt(path.format(i), delimiter=','))

    sub_corr = np.array(sub_corr)
    
    return sub_corr

In [ ]:
def plot_solution(solution = np.array, show_output = True, name = str):
    
    K = solution.shape[0]


    for i in range(0, K):
        fig, axes = plt.subplots(1, 2, sharex=True, figsize=(30,30))

        ax = sns.heatmap(sub_corr[i, :], ax=axes[0], center=0, vmin = -0.5, vmax = 0.5, 
                         square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
        ax.set_title('Covariance')

        ax = sns.heatmap(solution[i, :], ax=axes[1], center=0, vmin = -0.5, vmax = 0.5,
                         square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
        ax.set_title('Inverse Covariance')

        fig.savefig("plots/{0}_heatmap{1}.png".format(name, i))

In [ ]:
sub_corr_path = "/storage/groups/bds01/datasets/brains/sub_corr50/sub_corr{0}.csv"

sub_corr = read_sub_corr(K=50, path = sub_corr_path)

In [ ]:
sub_corr.shape

In [ ]:
start = 0
stop = 50

sub_corr = []

### on premisis
for i in range(start, stop):
    sub_corr.append(genfromtxt("../data/sub_corr50/sub_corr{0}.csv".format(i), delimiter=','))

    
sub_corr = np.array(sub_corr)
sub_corr.shape

# SGL

In [ ]:
N = 436
gamma = 0.3

lambda1_range = np.logspace(-0.3, -1, 10)

est_uniform, est_indv, statistics = K_single_grid(sub_corr, lambda1_range, N,
                                                  method = 'eBIC', gamma = gamma, 
                                                  latent = False, use_block = True)

In [ ]:
plot_solution(solution=est_uniform["Theta"], name="SGL")

In [ ]:
with open('statistics.txt', 'w') as f:
    print(statistics, file=f)

# GGL

In [ ]:
# N = 436
N = 50
gamma = 0.3

lambda1_range = np.logspace(0, -2, 8)
lambda2_range = np.logspace(-0.5, -2, 8)

In [ ]:
P_50 = glasso_problem(sub_corr[:50,:], N, reg_params = {'lambda1_range': lambda1_range, 'lambda2_range': lambda2_range}, 
                      latent = False, do_scaling = False)

In [ ]:
modelselect_params = {'lambda1_range': lambda1_range, 'lambda2_range': lambda2_range}

In [ ]:
P_50.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = gamma)

In [ ]:
sol = P_50.solution.precision_
sol.shape

In [ ]:
K = "50"

if not os.path.exists("../data/est_group{0}/".format(K)):
    os.mkdir("../data/est_group{0}/".format(K))

# dump matrices into csv
for i in range(start, stop):
    np.savetxt("../data/est_group{0}/Theta{1}.csv".format(K, i), sol[i], 
               delimiter=",", header='')
    
with open("../data/group_statistics{0}.txt".format(K), 'w') as f:
    print(P_50.modelselect_stats, file=f)

In [ ]:
statistics_50 = P_50.modelselect_stats

In [ ]:
statistics_50.keys()

In [ ]:
#plot for GGL
fig = surface_plot(statistics_50['L1'], statistics_50['L2'], statistics_50['BIC'][0.7], name = 'eBIC')
fig.show()

In [ ]:
plot_solution(solution=P_10.solution.precision_, name="SGL")

In [ ]:
with open('new_MGL_statistics.txt', 'w') as f:
    print(statistics_10, file=f)

In [ ]:
N = 436
gamma = 0.3

lambda1_range = [0.18957357, 0.14677993, 0.11364637]
lambda2_range = [0.01, 0.005, 0.001]
# lambda2_range = np.logspace(-2, -3, 5)

In [ ]:
P_10v2 = glasso_problem(sub_corr[:9,:], N, reg_params = {'lambda1_range': lambda1_range, 'lambda2_range': lambda2_range}, 
                      latent = False, do_scaling = False)

In [ ]:
modelselect_params = {'lambda1_range': lambda1_range, 'lambda2_range': lambda2_range}

In [ ]:
P_10v2.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = gamma)

In [ ]:
plot_solution(solution=P_10v2.solution.precision_, name="MGL")

In [ ]:
statistics_10v2 = P_10v2.modelselect_stats
statistics_10v2

In [ ]:
#plot for GGL
fig = surface_plot(statistics_10v2['L1'], statistics_10v2['L2'], statistics_10v2['BIC'][0.7], name = 'eBIC')
fig.savefig("plots/GGL_l1_l2_10v2.png")

In [ ]:
N = 436
gamma = 0.3

lambda1_range = [0.18957357, 0.14677993, 0.11364637]
lambda2_range = [0.001, 0.0005, 0.0000001]
# lambda2_range = np.logspace(-2, -3, 5)

In [ ]:
P_10v3 = glasso_problem(sub_corr[:9,:], N, reg_params = {'lambda1_range': lambda1_range, 'lambda2_range': lambda2_range}, 
                      latent = False, do_scaling = False)

In [ ]:
modelselect_params = {'lambda1_range': lambda1_range, 'lambda2_range': lambda2_range}

In [ ]:
P_10v3.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = gamma)

In [ ]:
statistics_10v3 = P_10v3.modelselect_stats
statistics_10v3

In [ ]:
#plot for GGL
fig = surface_plot(statistics_10v3['L1'], statistics_10v3['L2'], statistics_10v3['BIC'][0.7], name = 'eBIC')
fig.savefig("plots/GGL_l1_l2_10v3.png")